### Today we are going to perform the simple classification of the amazon reviews' sentiment.

### Please, download the dataset amazon_baby.csv.

In [173]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import time
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score


def remove_punctuation(text):
    import string
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

#https://query.data.world/s/5he2p6xnxkd3d32xnttrzuttt3yqvr?dws=00000 - link to dataset

from google.colab import drive
drive.mount('/content/drive')

shoes_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/amazon_uk_shoes_df.csv')
shoes_df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,url,product_name,reviewer_name,review_title,review_text,review_rating,verified_purchase,review_date,helpful_count,uniq_id,scraped_at
0,https://www.amazon.co.uk/dp/B07SBX32T5,Klasified Women's Transparent Clear Sneaker Sh...,Jocelyn McSayles,Love em,Love these. Was looking for converses and thes...,5.0,True,Reviewed in the United States on 2 June 2020,2 people found this helpful,36eae4e5-2894-5279-a0b7-d2b330e2b814,24/12/2021 02:26:25
1,https://www.amazon.co.uk/dp/B07SBX32T5,Klasified Women's Transparent Clear Sneaker Sh...,Kenia Rivera,The plastic ripped,"The shoes are very cute, but after the 2nd day...",2.0,True,Reviewed in the United States on 28 October 2021,NaN,f4778bb8-3070-5cb1-b5aa-ffce41a97b57,24/12/2021 02:26:25
2,https://www.amazon.co.uk/dp/B07SBX32T5,Klasified Women's Transparent Clear Sneaker Sh...,Chris Souza,Good quality,Good quality,5.0,True,Reviewed in the United States on 20 January 2021,NaN,db5a7525-d40b-5265-84d8-df4f29837a3b,24/12/2021 02:26:25
3,https://www.amazon.co.uk/dp/B07SBX32T5,Klasified Women's Transparent Clear Sneaker Sh...,Amazon Customer,Good,Great,5.0,True,Reviewed in the United States on 22 April 2021,NaN,75a42851-6462-54b5-988a-27d336221943,24/12/2021 02:26:25
4,https://www.amazon.co.uk/dp/B08SW434MG,"GUESS Women's Bradly Gymnastics Shoe, White, 7 UK",Graziella,PERFETTE!!,Ho scelto il modello bianco con rifinitura die...,5.0,True,Reviewed in Italy on 2 April 2021,2 people found this helpful,232dee43-849e-5d06-ba05-efb3f4814714,24/12/2021 02:26:25


In [174]:
shoes_df['review_text'] = shoes_df['review_text'].fillna('')

#short test:
shoes_df["review_text"][38] == shoes_df["review_text"][38]


#in order to remove punctuation, we need to remove NaNs first (replace with strings). Otherwise punctuation function will call exceptions for rows with NaNs, as they are special floating-point type (not string)


True

In [175]:
shoes_df['review_text'] = shoes_df['review_text'].apply(remove_punctuation)

#short test:
shoes_df['review_text'][15] == 'This is the second pair I bought Loved the first pair however not pleased with the 2nd pair There is no padding in the tongue part of the shoe and it is causing irritation to my skin where it rubs I am very disappointed and will not buy them again'
remove_punctuation(shoes_df['review_text'][15]) == 'This is the second pair I bought Loved the first pair however not pleased with the 2nd pair There is no padding in the tongue part of the shoe and it is causing irritation to my skin where it rubs I am very disappointed and will not buy them again'

#now we are removing punctuation


True

In [176]:
print("Max rating: ", max(shoes_df['review_rating']))
print("Min rating: ", min(shoes_df['review_rating']))

#we find out, that shoes are rated from 1 to 5, so we get rid of ratings equal 3, becouse they are neutral and wont be helpful in our assignment

shoes_df = shoes_df[shoes_df.review_rating != 3.0]

#short test:
sum(shoes_df["review_rating"] == 3.0)

#test was passed, they are no ratings equal 3


Max rating:  5.0
Min rating:  1.0


0

In [177]:
shoes_df['review_date'] = shoes_df['review_date'].str.split('on').str[0]
shoes_df['review_date'] = shoes_df['review_date'].str.split('in').str[1]

print(shoes_df['review_date'].unique())

shoes_df.rename(columns = {'review_date':'review_country'}, inplace = True)

#we check what are the countries that reviews were posted from, to extract reviews from english-spoken ones and rename the review date column for it to be more accurate

[' the United States ' ' Italy ' ' Germany ' ' Canada ' ' Japan '
 ' Mexico ' ' Spa' ' France ' ' Australia ' ' S' ' India ' ' Sweden '
 ' Brazil ' ' Poland ']


In [178]:
shoes_df = shoes_df[(shoes_df.review_country ==  ' the United States ') | (shoes_df.review_country ==  ' Canada ') | (shoes_df.review_country ==  ' Australia ')]

#we filter our dataframe by english-spoken countries reviews: the United States, Canada, Australia

In [179]:

shoes_df.loc[shoes_df["review_rating"] <= 2, "review_rating"] = -1
shoes_df.loc[shoes_df["review_rating"] >= 4, "review_rating"] = 1


#using loc to filter column and replace certain values, firstly we need to replace lower values to -1, becouse if we would like to transform high rating first, and set them to 1,
#second transformation of low ratings would turn all of the ratings to -1

#short test:
print(sum(shoes_df["review_rating"]**2 != 1))

#test is passed, because since we replaced all remaining ratings with 1s and -1s, their 2nd power wont result in anything else than 1s

print(sum(shoes_df["review_rating"] == 1))
print(sum(shoes_df["review_rating"] == -1))

#so we got 2863 good ratings and 685 bad ratings in total

0
2863
685


In [180]:
shoes_df.head(25)

,url,product_name,reviewer_name,review_title,review_text,review_rating,verified_purchase,review_country,helpful_count,uniq_id,scraped_at
0,https://www.amazon.co.uk/dp/B07SBX32T5,Klasified Women's Transparent Clear Sneaker Sh...,Jocelyn McSayles,Love em,Love these Was looking for converses and these...,1.0,True,the United States,2 people found this helpful,36eae4e5-2894-5279-a0b7-d2b330e2b814,24/12/2021 02:26:25
1,https://www.amazon.co.uk/dp/B07SBX32T5,Klasified Women's Transparent Clear Sneaker Sh...,Kenia Rivera,The plastic ripped,The shoes are very cute but after the 2nd day ...,-1.0,True,the United States,NaN,f4778bb8-3070-5cb1-b5aa-ffce41a97b57,24/12/2021 02:26:25
2,https://www.amazon.co.uk/dp/B07SBX32T5,Klasified Women's Transparent Clear Sneaker Sh...,Chris Souza,Good quality,Good quality,1.0,True,the United States,NaN,db5a7525-d40b-5265-84d8-df4f29837a3b,24/12/2021 02:26:25
3,https://www.amazon.co.uk/dp/B07SBX32T5,Klasified Women's Transparent Clear Sneaker Sh...,Amazon Customer,Good,Great,1.0,True,the United States,NaN,75a42851-6462-54b5-988a-27d336221943,24/12/2021 02:26:25
14,https://www.amazon.co.uk/dp/B07S1XM3L7,"adidas Women's Retrorun Shoes Running, Core Bl...",Lindsay,Perfect right outta the box,True to size If between Id probably go with yo...,1.0,True,Canada,One person found this helpful,b64632c5-6f24-51eb-9275-6614fed29f1a,24/12/2021 02:26:25
15,https://www.amazon.co.uk/dp/B07S1XM3L7,"adidas Women's Retrorun Shoes Running, Core Bl...",Lynn Maclean,Disappointed,This is the second pair I bought Loved the fir...,-1.0,True,Canada,NaN,b9c4399e-01de-5613-b597-915dc37944a7,24/12/2021 02:26:25
16,https://www.amazon.co.uk/dp/B07S1XM3L7,"adidas Women's Retrorun Shoes Running, Core Bl...",Martine Roy,Wow!,I love these shoes so much its my second pair ...,1.0,True,Canada,NaN,605fd069-17be-5f3f-9f6e-9ed6dc266b0f,24/12/2021 02:26:25
17,https://www.amazon.co.uk/dp/B0125TMZGK,"Aravon Women's Betty-AR Oxfords, Stone, 5.5 UK",Amazon Customer,Comfortable and attractive,I have hard to fit feet and often a wide fitti...,1.0,True,Canada,2 people found this helpful,bce0114a-c0fe-5472-bbb8-377cb21dc853,24/12/2021 02:26:25
19,https://www.amazon.co.uk/dp/B0125TMZGK,"Aravon Women's Betty-AR Oxfords, Stone, 5.5 UK",Amazon Customer,My right foot fits size 11 fine but not the sa...,I really need 11 12 but these shoes do not com...,1.0,True,Canada,NaN,0190a659-ae61-5c28-8b9d-65199697f02e,24/12/2021 02:26:25
21,https://www.amazon.co.uk/dp/B0125TMZGK,"Aravon Women's Betty-AR Oxfords, Stone, 5.5 UK",Burger Lover - Dalton,Great quality and comfort shoes,Great quality and comfort shoesI was so thrill...,1.0,True,the United States,2 people found this helpful,f1bafbd5-ff6a-52c7-8929-68bf9da9eda2,24/12/2021 02:26:25


## CountVectorizer
In order to analyze strings, we need to assign them numerical values. We will use one of the simplest string representation, which transforms strings into the $n$ dimensional vectors. The number of dimensions will be the size of our dictionary, and then the values of the vector will represent the number of appereances of the given word in the sentence.

In [181]:
vectorizer = CountVectorizer()
reviews_train_example = ["We like apples",
                   "We hate oranges",
                   "I adore bananas",
                   "We like like apples and oranges",
                   "They dislike bananas"]

X_train_example = vectorizer.fit_transform(reviews_train_example)

print(vectorizer.get_feature_names_out())
print(X_train_example.todense())



['adore' 'and' 'apples' 'bananas' 'dislike' 'hate' 'like' 'oranges' 'they'
 'we']
[[0 0 1 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 1 0 1]
 [1 0 0 1 0 0 0 0 0 0]
 [0 1 1 0 0 0 2 1 0 1]
 [0 0 0 1 1 0 0 0 1 0]]


In [182]:
reviews_test_example = ["They like bananas",
                   "We hate oranges bananas and apples",
                   "We love bananas"] #New word!

X_test_example = vectorizer.transform(reviews_test_example)

print(X_test_example.todense())

[[0 0 0 1 0 0 1 0 1 0]
 [0 1 1 1 0 1 0 1 0 1]
 [0 0 0 1 0 0 0 0 0 1]]


We should acknowledge few facts. Firstly, CountVectorizer does not take order into account. Secondly, it ignores one-letter words (this can be changed during initialization). Finally, for test values, CountVectorizer ignores words which are not in it's dictionary.

In [183]:

x = shoes_df["review_text"]
y = shoes_df["review_rating"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=44)

#splitting reviews and ratings into test and train data


In [184]:

vectorizer = CountVectorizer()

x_train_transformed = vectorizer.fit_transform(x_train)

print(vectorizer.get_feature_names_out())


['052821' '10' '100' ... 'étroite' 'été' 'único']


In [185]:

model = LogisticRegression(solver='sag', max_iter=200)
model.fit(x_train_transformed, y_train)

#creating model using train data

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(max_iter=200, solver='sag')

In [186]:

d = {'coefs':model.coef_[0],'values':vectorizer.get_feature_names_out()}
df = pd.DataFrame(d)

df = df.sort_values("coefs", ascending=False)

print("Positive words:")
print(df.head(10)["values"])
print("Negative words:")
print(df.tail(10)["values"])

#we zipped coefficients and corresponding values in dictionary and made a dataframe out of it to sort it descending by coefficient values

Positive words:
2468          great
1190    comfortable
3218           love
3922        perfect
1201          comfy
2425           good
3587           nice
6096           well
3222          loves
2548          happy
Name: values, dtype: object
Negative words:
4477           return
405           already
2559              has
2550             hard
480             apart
3637              not
1683     disappointed
5178          started
5806    uncomfortable
1951             even
Name: values, dtype: object


In [187]:

x_test_transformed = vectorizer.transform(x_test)

start = time.time()
y_pred = model.predict(x_test_transformed)
end = time.time()
modelpredicttime = end - start

#we transform test data using vectorizer we used to fit data and measure time of evaluation

In [188]:

start = time.time()
y_pred_proba = model.predict_proba(x_test_transformed)
end = time.time()
probapredicttime = end - start

print(model.classes_)

#classes we got predictions for are -1 and 1, for negative and positive reviews.

[-1.  1.]


In [189]:

d = {'predictions':y_pred_proba[:, 0],'values': x_test}
#we create a dictionary of probability that the review is negative and corresponding review
df = pd.DataFrame(d)

df = df.sort_values("predictions")

print("Positive reviews:")
print(df.head(5)["values"])
print("Negative reviews:")
print(df.tail(5)["values"])

#since we sorted the values ascending, we get smallest probablity values first, which means that probabilty of first reviews to be negative is really low,
#and the highest probability of reviews being negative is for the last ones

Positive reviews:
5041    The Ideal User for This ShoesI wore this pair ...
2379    I owned a pair of Merrell shoes that I wore fo...
3839    These are absolutely amazing I searched high a...
4097    Honestly the best water shoes I have ever purc...
6255    Great shoe Look great and feel great Go true t...
Name: values, dtype: object
Negative reviews:
6613    I am thinking about sending them back i put th...
3436    My wife loved the designcolor of the Propet Wo...
5837    They sewed the damn shoe laces into the back o...
4873    The back material and the front material insid...
1959    The back material and the front material insid...
Name: values, dtype: object


In [190]:

print(f"Model accuracy [{round(model.score(x_test_transformed, y_test), 2)}%], Time of evaluation: {modelpredicttime}")
print(f"Model accuracy [{round(accuracy_score(y_test, y_pred), 2)}%], Time of evaluation: {probapredicttime}")


Model accuracy [0.89%], Time of evaluation: 0.0007643699645996094
Model accuracy [0.89%], Time of evaluation: 0.0010929107666015625


In [191]:
significant_words = ['love','great','even','happy','comfortable','perfect','comfy','good','nice','well','loves','return','already','has','hard','apart','dissapointed','not','started', 'uncomfortable']

#we want to redo the whole process using shorter dictionary, containing only most posivite and most negative words

In [192]:


x = shoes_df["review_text"]
y = shoes_df["review_rating"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=44)

vectorizer_limited = CountVectorizer()
vectorizer_limited.fit_transform(significant_words)

#we create our vectorizer basing off limited dictionary

print(vectorizer_limited.get_feature_names_out())

x_train_transformed = vectorizer_limited.transform(x_train)
x_test_transformed = vectorizer_limited.transform(x_test)

#we transform our train and test data using limited vectorizer

model = LogisticRegression(solver='sag', max_iter=200)
model.fit(x_train_transformed, y_train)

#we create a model using transformed training data

start = time.time()
y_pred = model.predict(x_test_transformed)
end = time.time()
modelpredicttime = end - start
start = time.time()
y_pred_proba = model.predict_proba(x_test_transformed)
end = time.time()
probapredicttime = end - start

#we predict results for test data using created model and time measuring

d1 = {'coefs':model.coef_[0],'values':vectorizer_limited.get_feature_names_out()}
df1 = pd.DataFrame(d1)

df1 = df1.sort_values("coefs", ascending=False)

print("Positive words:")
print(df1.head(10)["values"])
print("Negative words:")
print(df1.tail(10)["values"])

d2 = {'predictions':y_pred_proba[:, 0],'values': x_test}
df2 = pd.DataFrame(d2)
df2 = df2.sort_values("predictions")

print("Positive reviews:")
print(df2.head(5)["values"])
print("Negative reviews:")
print(df2.tail(5)["values"])

#printing best and worst reviews

['already' 'apart' 'comfortable' 'comfy' 'dissapointed' 'even' 'good'
 'great' 'happy' 'hard' 'has' 'love' 'loves' 'nice' 'not' 'perfect'
 'return' 'started' 'uncomfortable' 'well']
Positive words:
3           comfy
15        perfect
7           great
8           happy
12          loves
2     comfortable
11           love
13           nice
19           well
6            good
Name: values, dtype: object
Negative words:
4      dissapointed
10              has
14              not
9              hard
5              even
1             apart
0           already
17          started
16           return
18    uncomfortable
Name: values, dtype: object
Positive reviews:
6255    Great shoe Look great and feel great Go true t...
459     I wear a womens size 12 US which generally tra...
2379    I owned a pair of Merrell shoes that I wore fo...
2228    These are by far the most comfortable golf sho...
557     I bought these shoes for casual comfy days  Do...
Name: values, dtype: object
Negative revie

In [193]:

print(df1)

#we can see that biggest positive impact is for words comfy, perfect, great, and biggest negative impact is for words uncomfortable, return and started.
#word started might be on the "negative" words list if for example mostly used in context like "started to hurt my feet" etc.

       coefs         values
3   1.875479          comfy
15  1.794219        perfect
7   1.462435          great
8   1.440261          happy
12  1.399331          loves
2   1.221617    comfortable
11  1.094659           love
13  0.984774           nice
19  0.905362           well
6   0.832784           good
4   0.182465   dissapointed
10 -0.621416            has
14 -0.806912            not
9  -0.855929           hard
5  -0.880807           even
1  -1.467294          apart
0  -1.531463        already
17 -1.613474        started
16 -1.673782         return
18 -2.474942  uncomfortable


In [194]:

print(f"Model accuracy [{round(model.score(x_test_transformed, y_test), 2)}%], Time of evaluation: {modelpredicttime}")
print(f"Model accuracy [{round(accuracy_score(y_test, y_pred), 2)}%], Time of evaluation: {probapredicttime}")

#Previous model data:
#Model accuracy [0.89%], Time of evaluation: 0.0007226467132568359
#Model accuracy [0.89%], Time of evaluation: 0.0009052753448486328
#Model accuracy for the limited dictionary is lower for 4% in comparison with the previous model, but the evaluation time decreased a little bit in both cases. It is caused by the fact that we need to do less calculations for more limited dictionary

Model accuracy [0.85%], Time of evaluation: 0.00038886070251464844
Model accuracy [0.85%], Time of evaluation: 0.0003402233123779297


In [226]:
#finding best items by 2 best words and worst items by 2 worst words and checking if avg rating of these items matching the results
#best items

best_items = []
total_good_rating = 0
counter = 0
good_items_counter = 0

print("Best items: ")
for index, item in shoes_df['product_name'].items():
  if ("comfy" in shoes_df['review_text'][index]) and ("perfect" in shoes_df['review_text'][index]):
    total_good_rating += shoes_df['review_rating'][index]
    counter += 1
    if item not in best_items:
      best_items.append(item)
      print(item)
      good_items_counter += 1

print("Number of items: ", good_items_counter)
print("Average rating for good items: ", total_good_rating/counter)

#worst items

worst_items = []
total_bad_rating = 0
counter = 0
bad_items_counter = 0

print("Worst items: ")
for index, item in shoes_df['product_name'].items():
  if (item not in worst_items) and (("uncomfortable" in shoes_df['review_text'][index]) and ("return" in shoes_df['review_text'][index])):
    total_bad_rating += shoes_df['review_rating'][index]
    counter += 1
    if item not in worst_items:
      worst_items.append(item)
      print(item)
      bad_items_counter += 1


print("Number of items: ", bad_items_counter)
print("Average rating for good items: ", total_bad_rating/counter)


#while for reviews containing positive words all of them appeared to be positive, not all the reviews containing both words "uncomfortable" and "return" appeared to be negative.
#it my be caused by the context, if these words were used for example in such a sentence "I thought these shoes would be uncomfortable and I was going to return them [...]"

Best items: 
Reef Little Ahi Convertible Kids Sandals UK 5-6 Leopard
New Balance Men's Cruz V2 Fresh Foam Sneaker, Black/Castlerock/Nubuck, 6.5 UK
Skechers Women's City PRO-Easy Moving Sneaker, TPE, 7 M US
Linodes (Tent Leather Upper Jazz Shoe Slip-on for Women and Men's Dance Shoes brown Size: 7 Women/6.5 Men
Chaco Men&#39;s Z1 Classic Athletic Sandal
Chaco Women's Odyssey blue Size: 10 UK
adidas Terrex CC Boat Shoes Mens
Merrell Women's Hydro Moc Water Shoe, Black, 7.5 UK
Allonsi | Thomas Mens Genuine Leather Chelsea Boots | Goodyear Welt Construction | Classic Chelsea Boots | Leather Sole | Handcrafted Luxury Leather Shoes Brown Size: 13
Propet Women's W3841 Sneaker, Black Suede, 4.5 UK
Rocket Dog Women's GIAPS-001 Sneaker, Black, 9 UK
Reebok Women's Cloudride DMX 4.0 Walking Shoe, Black/Grey/Cyan/White, 7.5 M US
konhill Women's Casual Walking Shoes Breathable Mesh Work Slip-on Sneakers, 0701 All White, 5
ASICS - Mens Solution Speed Ff Shoes, 8 UK, Cherry Tomato/Black
adidas Women's